In [4]:
import praw
import pandas as pd
import datetime
import concurrent.futures

reddit = praw.Reddit(client_id='_37znztIqSaRtA8ZWOCRXg', client_secret="0wz5rUMlMjJBSCwUuXhuDt_64OOCyA",
                     user_agent='CIS509')

subreddit = reddit.subreddit('DoorDash')

def process_submission(submission):
    try:
        submission.comments.replace_more(limit=0)
        comments_data = []
        for comment in submission.comments.list():
            timestamp = datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            comments_data.append([comment.id,
                                 comment.author,
                                 timestamp,
                                 comment.parent_id,
                                 submission.id,
                                 submission.title,
                                 comment.body,
                                 comment.permalink])
        return comments_data
    except Exception as e:
        print(f"Error processing submission with id {submission.id}: {e}")
        return []

comments_data = []
count = 0
with concurrent.futures.ThreadPoolExecutor() as executor:
    submission_futures = [executor.submit(process_submission, submission) for submission in subreddit.new(limit=1000)]
    for submission_future in concurrent.futures.as_completed(submission_futures):
        comments_data += submission_future.result()
        count += len(submission_future.result())
        print(f"Processed {count} comments.")
        if count >= 2000:
            break

comments_df = pd.DataFrame(comments_data, columns=['Comment ID', 'Author', 'Timestamp',
                                                    'Comment is a reply to', 'Comment Thread ID',
                                                    'Thread Title', 'Comment Body', 'Comment Permalink'])

print(comments_df)


Processed 19 comments.
Processed 47 comments.
Processed 52 comments.
Processed 240 comments.
Processed 251 comments.
Processed 254 comments.
Processed 261 comments.
Processed 262 comments.
Processed 316 comments.
Processed 338 comments.
Processed 339 comments.
Processed 342 comments.
Processed 345 comments.
Processed 345 comments.
Processed 557 comments.
Processed 569 comments.
Processed 569 comments.
Processed 577 comments.
Processed 580 comments.
Processed 582 comments.
Processed 582 comments.
Processed 598 comments.
Processed 601 comments.
Processed 603 comments.
Processed 625 comments.
Processed 644 comments.
Processed 690 comments.
Processed 692 comments.
Processed 694 comments.
Processed 702 comments.
Processed 706 comments.
Processed 713 comments.
Processed 1189 comments.
Processed 1197 comments.
Processed 1209 comments.
Processed 1213 comments.
Processed 1214 comments.
Processed 1252 comments.
Processed 1258 comments.
Processed 1271 comments.
Processed 1271 comments.
Processed 

In [5]:
comments_df.to_csv('DoorDash_comments_21_03.csv',index=False)

In [6]:
!pip install textblob

In [7]:
!pip install nltk

In [8]:
!pip install pandas

In [9]:
import pandas as pd
data = pd.read_csv('DoorDash_comments_21_03.csv')
data.head(2)

,Comment ID,Author,Timestamp,Comment is a reply to,Comment Thread ID,Thread Title,Comment Body,Comment Permalink
0,jd1eki0,renbutler2,2023-03-20 20:06:54,t3_11x4f2x,11x4f2x,Leave it at the door,There's a simple answer to all questions like ...,/r/doordash/comments/11x4f2x/leave_it_at_the_d...
1,jd1dwcf,honorablefish,2023-03-20 20:01:17,t3_11x4f2x,11x4f2x,Leave it at the door,Never,/r/doordash/comments/11x4f2x/leave_it_at_the_d...


In [10]:
#Parse csv's to only extract column called 'message'
messages = data['Comment Body']

In [11]:
from textblob import TextBlob
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
#Text pre-processing functions
def text_processing(message):
    
    #Generating the list of words in the message (hastags and other punctuations removed) and convert to lowercase
    def form_sentence(message):
        message = message.lower() #Make messages lowercase 
        message_blob = TextBlob(message.lower()) #Convert to 'textblob' which provides a simple API for NLP tasks
        return ' '.join(message_blob.words)
    new_message = form_sentence(message)
    
    #Removing stopwords and words with unusual symbols
    def no_user_alpha(message):
        message_list = [item for item in message.split()] 
        clean_words = [word for word in message_list if re.match(r'[^\W\d]*$', word)] #remove punctuation and strange characters
        clean_sentence = ' '.join(clean_words) 
        clean_mess = [stopword for stopword in clean_sentence.split() if stopword not in stopwords.words('english')] #remove stopwords
        return clean_mess
    no_punc_message = no_user_alpha(new_message)
    
    #Normalizing the words in messages 
    def normalization(message_list):
        lem = WordNetLemmatizer()
        normalized_message = []
        for word in message_list:
            normalized_text = lem.lemmatize(word,'v') #lemmatize words
            normalized_message.append(normalized_text)
        return normalized_message
    return normalization(no_punc_message)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
#Print to console and write to file
f = open('processed_text_doordash.csv','w', encoding='utf8')
for message in messages: 
    message = text_processing(message)
    for term in message:
        f.write(term+" ")
    f.write("\n")

In [15]:
f.close()

In [2]:
!pip install spacy

  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached pathy-0.10.1-py3-none-any.whl (48 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached wasabi-1.1.1-py3-none-any.whl (27 kB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
  Using cached confection-0.0.4-py3-none-any.whl (32 kB)
  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=7681 sha256=4c35111fe5262956ece6b198f37da8690b179b043501e80d206a28e5d4c1aed4
  Stored in directory: c:\users\mkusumur\appdata\local\pip\cache\wheels\41\11\53\911724983aa48deb94792432e14e518447212dd6c5477d49d3
Successfully built contextvars


In [3]:
!python -m spacy download en_core_web_lg

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
!pip install pydantic --upgrade

In [5]:
!pip install blis

In [6]:
!pip install thinc

In [32]:
!pip install typing_extensions>=3.7.4

In [12]:
import spacy
nlp = spacy.load("en_core_web_lg")
docs = list(nlp.pipe(data['Comment Body']))

In [13]:
import operator
np_dict = {}
for doc in docs:
    for np in doc.noun_chunks:
        np_dict[np.text]=np_dict.get(np.text,0)+1
sortedDict = sorted(np_dict.items(),key=operator.itemgetter(1),reverse=True)
print(sortedDict)

[('I', 960), ('it', 747), ('you', 433), ('that', 204), ('the watch', 146), ('i', 115), ('It', 107), ('me', 90), ('they', 82), ('this', 69), ('🔥', 66), ('which', 57), ('You', 54), ('what', 53), ('the app', 46), ('them', 41), ('Thanks', 41), ('That', 40), ('Huawei', 40), ('Strava', 31), ('https://celest-watches.com', 31), ('46mm', 28), ('everything', 25), ('More watch', 25), ('HUAWEI WATCH', 25), ('mine', 22), ('something', 22), ('the phone', 22), ('This', 22), ('notifications', 21), ('apps', 21), ('\\-', 21), ('iOS', 20), ('anything', 20), ('we', 19), ('the US', 18), ('my watch', 18), ('GT3', 16), ('a lot', 16), ('Android', 15), ('my phone', 15), ('Garmin', 14), ('time', 13), ('What', 13), ('the screen', 13), ('workouts', 13), ('all', 12), ('your watch', 12), ('people', 12), ('those', 12), ('the problem', 11), ('the GT3', 11), ('They', 11), ('gt3', 11), ('your phone', 11), ('ECG', 11), ('a map', 11), ('nothing', 10), ('the battery', 10), ('things', 10), ('Huawei Health', 10), ('GT2', 10

In [14]:
entity_dict = {}
for doc in docs:
    for entity in doc.ents:
        entity_dict[entity.text]=entity_dict.get(entity.text,0)+1
sortedDict = sorted(entity_dict.items(),key=operator.itemgetter(1),reverse=True)
print(sortedDict)

[('Huawei', 111), ('3', 67), ('GT', 55), ('one', 41), ('Strava', 33), ('46mm', 30), ('Huawei Health', 28), ('Garmin', 27), ('first', 25), ('US', 23), ('2', 22), ('GT2', 22), ('Pro', 18), ('UK', 17), ('Google', 15), ('gt3', 14), ('android', 13), ('Health', 13), ('GT Runner', 11), ('watchface', 11), ('China', 10), ('Compatible', 9), ('4', 9), ('third', 8), ('two', 8), ('5', 8), ('Android', 8), ('Samsung', 7), ('Europe', 7), ('Germany', 6), ('Bluetooth', 6), ('today', 6), ('Huawei Health App', 6), ('half', 6), ('6', 6), ('42mm', 6), ('3 Pro', 6), ('AOD', 6), ('100%', 5), ('2e', 5), ('\U0001f929', 5), ('\U0001f929\U0001f929', 5), ('Health Sync', 5), ('the day', 5), ('3rd', 5), ('HONOR MagicWatch', 5), ('SE', 5), ('Canada', 5), ('USA', 5), ('13', 5), ('Amazon', 5), ('7', 4), ('AppGallery', 4), ('Amazfit', 4), ('HRV', 4), ('\\-', 4), ('GT Cyber', 4), ('HMS', 4), ('huawei', 4), ('daily', 4), ('5 days', 4), ('Fitbit', 4), ('Apple', 4), ('GT3', 4), ('Pixel', 4), ('Sapphire', 4), ('tomorrow', 3)

In [16]:
# To get SpaCy tag list :https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/
from collections import Counter
verbs = []
for doc in docs:
    for token in doc: 
        if (not token.is_stop and not token.is_punct and token.pos_=="VERB"):
            verbs.append(token.lemma_)
verb_freq = Counter(verbs)
common_verbs = verb_freq.most_common(10)
print(common_verbs)

[('work', 116), ('watch', 79), ('think', 74), ('get', 72), ('use', 70), ('try', 69), ('find', 63), ('look', 59), ('know', 57), ('face', 52)]
